In [12]:


import numpy as np
from scipy.optimize import minimize

# Fonction de coût courant Ln(x, u)
def L(n, x, u):
    """
    Calcule le coût courant à l'étape n.
    Arguments : 
    - n : étape temporelle
    - x : état courant
    - u : contrôle (réel)

    Retourne : coût courant
    """
    return u ** 2 - x -1

# Fonction de récurrence fn(x, u)
def f(n, x, u):
    """
    Calcule l'état suivant selon la récurrence f.
    Arguments :
    - n : étape temporelle
    - x : état courant
    - u : contrôle (réel)

    Retourne : état suivant
    """
    return x + u

# Fonction de coût terminal g(x)
def g(x):
    """
    Calcule le coût terminal.
    Argument :
    - x : état final

    Retourne : coût terminal
    """
    return 0

# Optimisation continue pour chaque étape
def optimize_continuous(L_func, f_func, V_next, n, x):
    """
    Minimise la fonction de coût en continu sur l'ensemble des actions réelles (u).
    Arguments :
    - L_func : fonction de coût courant
    - f_func : fonction de récurrence
    - V_next : fonction valeur de l'étape suivante
    - n : étape temporelle
    - x : état courant

    Retourne : coût minimum et contrôle optimal u
    """
    # Fonction à minimiser : L(n, x, u) + V(n+1, f(n, x, u))
    def objective(u):
        return L_func(n, x, u) + V_next(f_func(n, x, u))[0]

    # Minimisation sur l'ensemble des réels (u)
    result = minimize(objective, x0=0)  # On part d'une estimation initiale u = 0
    optimal_u = result.x[0]  # Le contrôle optimal
    optimal_cost = result.fun  # Le coût minimal
    return optimal_cost, optimal_u

# Fonction principale d'induction rétrograde
def backward_induction(x0, time_horizon):
    """
    Résout le problème d'optimisation dynamique par induction rétrograde.
    Arguments :
    - x0 : état initial
    - time_horizon : horizon temporel (nombre d'étapes)

    Retourne : dictionnaire des fonctions valeur et politique optimale.
    """
    V = {}  # Stocker la fonction valeur pour chaque étape
    policy = {}  # Stocker la politique optimale (u optimal) pour chaque étape

    # Calcul de V(3, x) = inf(L3(x,u) + g(f3(x,u)))
    def V_3(x):
        return optimize_continuous(L, f, lambda x: (g(x), None), 3, x)

    # Calcul de V(2, x) = inf(L2(x,u) + V(3, f2(x,u)))
    def V_2(x):
        return optimize_continuous(L, f, V_3, 2, x)

    # Calcul de V(1, x) = inf(L1(x,u) + V(2, f1(x,u)))
    def V_1(x):
        return optimize_continuous(L, f, V_2, 1, x)

    # Calcul de V(0, x) = inf(L0(x,u) + V(1, f0(x,u)))
    def V_0(x):
        return optimize_continuous(L, f, V_1, 0, x)

    # Stockage des fonctions valeurs et des politiques optimales à chaque étape
    V[3] = V_3(x0)
    V[2] = V_2(x0)
    V[1] = V_1(x0)
    V[0] = V_0(x0)

    # Stockage des contrôles optimaux (politique) pour chaque étape
    policy[3] = V_3(x0)[1]
    policy[2] = V_2(x0)[1]
    policy[1] = V_1(x0)[1]
    policy[0] = V_0(x0)[1]

    return V, policy

# Utilisation de la récurrence pour calculer les états à chaque étape
def compute_states(x0, policy):
    """
    Calcule les états à chaque étape en utilisant la relation de récurrence.
    Arguments :
    - x0 : état initial
    - policy : dictionnaire des contrôles optimaux (politique)

    Retourne : liste des états successifs.
    """
    x_values = [x0]
    for t in range(4):  # Calcul des états de x1 à x4
        next_x = f(t, x_values[-1], policy[t])
        x_values.append(next_x)
    return x_values

# Paramètres
x0 = 0  # État initial
time_horizon = 4  # Horizon temporel

# Résolution du problème par induction rétrograde
V, policy = backward_induction(x0, time_horizon)

# Affichage des fonctions valeur et des politiques optimales
print("Fonctions valeur pour chaque étape :")
for t in range(4):
    print(f"V({t}, x0) = {V[t][0]}, u optimal = {policy[t]}")

# Calcul et affichage des états successifs (x1, x2, x3, x4)
x_values = compute_states(x0, policy)
print("\nÉtats calculés :")
for t, x in enumerate(x_values):
    print(f"x{t} = {x}")

Fonctions valeur pour chaque étape :
V(0, x0) = -7.499999999999999, u optimal = 1.5000000141634802
V(1, x0) = -4.25, u optimal = 1.00000000944232
V(2, x0) = -2.25, u optimal = 0.4999999943267469
V(3, x0) = -1.0, u optimal = 0.0

États calculés :
x0 = 0
x1 = 1.5000000141634802
x2 = 2.5000000236058
x3 = 3.000000017932547
x4 = 3.000000017932547
